In [0]:

print("Start")

In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
from datetime import datetime


# Snowflake Production
sfOptions = {
  "sfURL" : "appannie_aa_int_prod.us-east-1.snowflakecomputing.com",
  "sfUser" : "app_bdp_data_validation_qa",
  "sfPassword" : "0HN#s@Wa5$1R8jVj",
  "sfDatabase" : "AA_INTELLIGENCE_PRODUCTION",
  "sfSchema" : "ADL_STORE_PAID",
  "sfWarehouse" : "wh_dod_read7"
}
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"


start_date = '2021-04-01'
end_date = '2021-06-20'

print(spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  f"select distinct date from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where country_code='WW' and date>='{start_date}' and date<='{end_date}';") \
    .load().count())
    
print(spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  f"select distinct date from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where country_code!='WW' and date>='{start_date}' and date<='{end_date}';") \
    .load().count())

In [0]:


start_date = '2021-04-01'
end_date = '2021-06-20'

# Sum of metrics for WW
ww_sum = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  f"select sum(est_download) as v1, sum(est_paid_download) as v2, sum(est_organic_download) as v3, \
        sum(est_paid_search_download) as v4, sum(est_paid_in_app_ads_download) as v5, \
        sum(est_organic_search_download) as v6, sum(est_organic_featured_download) as v7\
        from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE \
        where date>='{start_date}' and date<='{end_date}' and country_code='WW';") \
    .load()
    
ww_sum.show()
        
# Sum of metrics for other countries
o_sum = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  f"select sum(est_download) as v1, sum(est_paid_download) as v2, sum(est_organic_download) as v3, \
        sum(est_paid_search_download) as v4, sum(est_paid_in_app_ads_download) as v5, \
        sum(est_organic_search_download) as v6, sum(est_organic_featured_download) as v7\
        from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE \
        where date>='{start_date}' and date<='{end_date}'  and country_code!='WW';") \
    .load()
    
o_sum.show()

In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from datetime import date
import random


start = date(2021, 4, 1)
end = date(2021, 6, 20)

for i in random.sample(range(start.toordinal(), end.toordinal()), 20):
    current = str(date.fromordinal(i))
    
    # Sum of metrics for WW
    ww_sum = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select sum(est_download) as v1, sum(est_paid_download) as v2, sum(est_organic_download) as v3, \
            sum(est_paid_search_download) as v4, sum(est_paid_in_app_ads_download) as v5, \
            sum(est_organic_search_download) as v6, sum(est_organic_featured_download) as v7\
            from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE \
            where date='{current}' and country_code='WW';") \
        .load()
        
    # Sum of metrics for other countries
    o_sum = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select sum(est_download) as v1, sum(est_paid_download) as v2, sum(est_organic_download) as v3, \
            sum(est_paid_search_download) as v4, sum(est_paid_in_app_ads_download) as v5, \
            sum(est_organic_search_download) as v6, sum(est_organic_featured_download) as v7\
            from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE \
            where date='{current}' and country_code!='WW';") \
        .load()
        
    
    ww = ww_sum.collect()[0]
    o = o_sum.collect()[0]
    if ww.V1 != ww.V1:
        print(f"Incorrect EST_DOWNLOAD Sum on {current}")
        
    print(f"{current}: {ww.V2-o.V2}, {ww.V3-o.V3}, {ww.V4-o.V4}, {ww.V5-o.V5}, {ww.V6-o.V6}, {ww.V7-o.V7}")

print("Checked.")


In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from datetime import date
import random


dates = [date(2021, 4, 1), date(2021, 4, 24), date(2021, 5, 29)]
product_keys = [20600012501710, 20600000013820, 20600010817872]

for i in range(len(dates)):
    d = str(dates[i])
    p = product_keys[i]
    
    # Sum of metrics for WW
    ww_sum = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select sum(est_download), sum(est_paid_download), sum(est_organic_download), \
            sum(est_paid_search_download), sum(est_paid_in_app_ads_download), \
            sum(est_organic_search_download), sum(est_organic_featured_download) \
            from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE \
            where date='{d}' and product_key='{p}' and country_code='WW';") \
        .load()
        
    # Sum of metrics for other countries
    o_sum = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select sum(est_download), sum(est_paid_download), sum(est_organic_download), \
            sum(est_paid_search_download), sum(est_paid_in_app_ads_download), \
            sum(est_organic_search_download), sum(est_organic_featured_download) \
            from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE \
            where date='{d}' and product_key='{p}' and country_code!='WW';") \
        .load()
        
    ww_sum.show()
    o_sum.show()
